In [79]:
import string
import re
import os
import pandas as pd
import csv

In [80]:
tg_path="."
tg_name="TG263_Nomenclature_Worksheet_20170815(TG263 v20170815).csv"
df = pd.read_csv(os.path.join(tg_path,tg_name))
tg_names = df['TG263-Primary Name'].to_list()
tg_names_rev = df['TG-263-Reverse Order Name'].to_list()

In [108]:
target_df = pd.read_csv('/mnt/iDriveShare/Kayla/unique_targets.csv')
target_names = target_df['Name'].to_list()
target_inst = target_df['Instances'].to_list()
target_len = target_df['Length'].to_list()

# with open('/mnt/iDriveShare/Kayla/unique_targets.csv', 'r') as f:
#     list_targets = [row[0] for row in csv.reader(f)] 

In [61]:
name = 'PTV_5000'
# list_names = ['CTV_6000','CTV_7000','GTV','CTV_5600','PTV_7000-05','PTV!_6000-05',
#               'PTV_All','PTV','CTV','PTV_ALL','PTV70+PTV60','PTV60','PTV56','CTV60','CTV70','PTV!_Eval_6000','PTV2n',
#               'CTVn_6000','CTVn','opt_CTV56_crop03','PTV^Physician1','GTV_2000^ICG','PTV1','GTVp1','GTVp2','PTVn','PTVsb']
list_names = ['CT 60','GTV','CTV_5600','PTV_7000-05','PTV_All','PTV70+PTV60','PTV60','PTV!_Eval_6000','PTV2n',
              'CTVn_6000','CTVn','opt_CTV56_crop03','PTV^Physician1','GTV_2000^ICG','PTV1','GTVp1','GTVp2','PTVn','PTVsb']

# note, can easily check spaces and legnth

# to do: add exmples from the TG 263 paper

In [103]:
# note, this does not work for rule 10 of the guidelines 
# TO DO: split by ^ immediately (after length and space check) and disregard everything afterwards
# thought process for suffixes: go through and remove correct parts of name in the correct ordre. 
#                               if anything is leftover without a ^, then it is wrong

# TO DO: 
# E-PTV_Ev05_xxxx	E-PTV_Ev05_xxxx
# E-PTV_xxxx	E-PTV_xxxx

def check_target_compliance(target_name):
    debug = False
    reason = ''
    #first check if there are spaces, then it's automatically not tg 263 compliant
    if ' ' in target_name:
        reason = "spaces"
        return False, reason

    if len(target_name) > 16:
        reason = "> 16 characters"
        return False, reason
    
    target_prefix = target_name.split("_")[0]
    target_suffix = target_name.replace(target_prefix,'')
    
    if debug:
        print("****")

        print("prefix", target_prefix)
#     print("suffix", target_suffix)

    is_correct = True
    reasoning = ''
  

    # CHECKING PREFIX
    list_allowed_prefixes = ['PTV!','GTV','CTV','ITV','IGTV','ICTV','PTV'] #rule 1
    list_allowed_classifiers = ['n', 'p', 'sb', 'par', 'v', 'vas',''] # rule 2, including none
    #note: ordering of the above mattes, PTV! sb before PTV, and '' should be last -- this is for when removing it from prefix
    
    if not target_prefix.startswith(tuple(list_allowed_prefixes)): # rule 1
        reason = "fails rule 1"
        return False, reason
    
    for p in list_allowed_prefixes: 
        if target_prefix.startswith(p):
            target_prefix = target_prefix.replace(p,'')
            break
   
    if debug:
        print("After Rule 1:",target_prefix)

    
    
    
    if target_prefix != '':
        if target_prefix[0] == "^": # rule 9
            if debug:
                print("its true, all after ^")
            return True, reason
        prefix_no_digits = target_prefix.rstrip(string.digits)
        if debug:
            print("prefix no digits:",prefix_no_digits)
        if not prefix_no_digits in list_allowed_classifiers: #rule 2
            reason = "Fails rule 2"
            return False, reason
        
        for c in list_allowed_classifiers:
            if target_prefix.startswith(c):
                target_prefix = target_prefix.replace(c,'')
                break
        if debug:
            print("After rule 2:",target_prefix)
        
        if target_prefix != '' and (len(target_prefix) > 1 or not target_prefix.isdigit()):
            # rule #3 --> could be tricky, saying len sb max 1 becuase i doubt the bunbers would go up to 10
            #but... there could be a random number there that isn't meant to represnet spatially distinct targets.
            reason = "fails rule 3"
            return False, reason
            
    # TO DO -- check if ends with -xx for prefix eg CTVp2-05
    
    # CHECKING SUFFIX
    
    # TO DO: SKIPPING RULES 4 AND 5 FOR NOW
    # for rule 5, check thorugh TG again, however will need to do starts with i believe to accoutn for extra dose etc put at end
    # then need to remove it and analyse rest of structure as before
    # rule #6
    
    if target_suffix != '':
        relative_dose_suffixes = ["_High", "_Mid","_Low"]
        if debug:
            print("suffix", target_suffix)

        numerical_dose = False
        if target_suffix.startswith(tuple(relative_dose_suffixes)):
            #ok it is relavice dose
            # TO DO: add 01 02 etc
            for r in relative_dose_suffixes:
                if target_suffix.startswith(r):
                    target_suffix = target_suffix.replace(r,'')
                    break

            placeholder = 1

        elif bool(re.match(r'^_\d{4}', target_suffix)): #cGy
            target_suffix = target_suffix[5:]
            placeholder = 2
            numerical_dose = True
    #     elif : TO DO : Gy

        if debug:
            print("After rule 6: ", target_suffix)
        if numerical_dose: #rule 7, this is optional, only if the # fx are indicated it should be with an x
            fraction_pattern = r'^x\d{1,2}'
            if bool(re.match(fraction_pattern, target_suffix)):
                fractions = True
                target_suffix = re.sub(fraction_pattern, '', target_suffix)

            if debug:
                print("After rule 7: ", target_suffix)

        #rule 8
        if bool(re.match(r'^-\d{2}', target_suffix)):
            target_suffix = target_suffix[3:]

            if debug:
                print("after rule 8:", target_suffix)

        # rule 9
    if target_suffix == '' or target_suffix[0] == "^":
#             print("ALL GOODDDDDD")
        return True, reason
    else:
#             print("FAILED~!", target_suffix)
        reason = "Suffix leftover: " + target_suffix
        return False, reason

            

In [110]:
for name in list_names:
#     print(name)
    passes, reason = check_target_compliance(name)
    print(name, passes, reason)

CTV_6000 True 
CTV_7000 True 
GTV True 
CTV_5600 True 
PTV_7000-05 True 
PTV!_6000-05 True 
PTV!_5600-05 True 
zPTV7000+PTV6000 False fails rule 1
Cavity_Oral-PTV False fails rule 1
PTV_All False Suffix leftover: _All
PTV True 
CTV True 
PTV_ALL False Suffix leftover: _ALL
PTV70+PTV60 False Fails rule 2
PTV60 False fails rule 3
PTV56 False fails rule 3
CTV60 False fails rule 3
CTV70 False fails rule 3
GTV70 False fails rule 3
CTV56 False fails rule 3
PTV70 False fails rule 3
CTV60 FC False spaces
PTV60 FC False spaces
PTV60+PTV70 False Fails rule 2
GTVn True 
CTVp True 
PTV_56 False Suffix leftover: _56
GTVp True 
CTV_70 False Suffix leftover: _70
CTV_56 False Suffix leftover: _56
PTV_70 False Suffix leftover: _70
GTV RT NECK False spaces
CTV62 RT NECK False spaces
MOD1 PTV62/30 False spaces
MOD1 PTV52/30 False spaces
MOD1 PTV60/30 False spaces
PTV ALL 30fx False spaces
PTV70+PTV66 False Fails rule 2
CTV66 False fails rule 3
PTV66 False fails rule 3
PTV70+66+60 False Fails rule 2
MOD1_

In [111]:
#TO DO: check for eg if Cavity-Oral-PTV is ok
pass_list = []
reason_list = []
for name in target_names:
#     print(name)
    passes, reason = check_target_compliance(name)
    pass_list.append(passes)
    reason_list.append(reason)
    
    print(name, passes, reason)

CTV_6000 True 
CTV_7000 True 
GTV True 
CTV_5600 True 
PTV_7000-05 True 
PTV!_6000-05 True 
PTV!_5600-05 True 
zPTV7000+PTV6000 False fails rule 1
Cavity_Oral-PTV False fails rule 1
PTV_All False Suffix leftover: _All
PTV True 
CTV True 
PTV_ALL False Suffix leftover: _ALL
PTV70+PTV60 False Fails rule 2
PTV60 False fails rule 3
PTV56 False fails rule 3
CTV60 False fails rule 3
CTV70 False fails rule 3
GTV70 False fails rule 3
CTV56 False fails rule 3
PTV70 False fails rule 3
CTV60 FC False spaces
PTV60 FC False spaces
PTV60+PTV70 False Fails rule 2
GTVn True 
CTVp True 
PTV_56 False Suffix leftover: _56
GTVp True 
CTV_70 False Suffix leftover: _70
CTV_56 False Suffix leftover: _56
PTV_70 False Suffix leftover: _70
GTV RT NECK False spaces
CTV62 RT NECK False spaces
MOD1 PTV62/30 False spaces
MOD1 PTV52/30 False spaces
MOD1 PTV60/30 False spaces
PTV ALL 30fx False spaces
PTV70+PTV66 False Fails rule 2
CTV66 False fails rule 3
PTV66 False fails rule 3
PTV70+66+60 False Fails rule 2
MOD1_

In [112]:

with open("unique_list_targets.csv","w") as f:
    writer = csv.writer(f)
    writer.writerow(["In-House Name","Instances","Length","Matches TG-263","Reason","Structure Type","Rules"])
    writer.writerows(zip(target_names, target_inst, target_len,pass_list,reason_list))


In [12]:
prefix = name.split("_")[0]

In [13]:
suffix = name.replace(prefix,'')

In [15]:
print("prefix",prefix)
print("suffix",suffix)

prefix PTV
suffix _5000


In [67]:
target_suffix = "x20x50"

fraction_pattern = r'^x\d{1,2}'
if bool(re.match(fraction_pattern, target_suffix)):
    fractions = True
    target_suffix = re.sub(fraction_pattern, '', target_suffix)


In [68]:
target_suffix

'x50'